In [1]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

/anaconda3/envs/dlcv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
IMG_HELP_PATH = '/Users/shankar/dev/code/ds/studies/data_science/dlcv'
import os
import sys
sys.path.append(os.path.abspath(IMG_HELP_PATH))
from common.preprocessing import ImageToArrayPreprocessor
from common.preprocessing import AspectAwarePreprocessor
from common.datasets import SimpleDatasetLoader
from common.nn.conv import MiniVGGNet

In [7]:
DATASET = '../datasets/flowers17/images'

In [8]:
print("[INFO] Loading images")
imagePaths = list(paths.list_images(DATASET))
classNames = [ pt.split(os.path.sep)[-2] for pt in imagePaths]
classNames = [str(x) for x in np.unique(classNames)]

[INFO] Loading images


In [9]:
classNames

['bluebell',
 'buttercup',
 'coltsfoot',
 'cowslip',
 'crocus',
 'daffodil',
 'daisy',
 'dandelion',
 'fritillary',
 'iris',
 'lilyvalley',
 'pansy',
 'snowdrop',
 'sunflower',
 'tigerlily',
 'tulip',
 'windflower']

In [10]:
aap = AspectAwarePreprocessor(64, 64)
iap = ImageToArrayPreprocessor()

In [11]:
sdl = SimpleDatasetLoader(preprocessors=[aap, iap])
(data, labels) = sdl.load(imagePaths, verbose=500)
data = data.astype("float")/255.0

[INFO] processed 500/1360
[INFO] processed 1000/1360


In [12]:
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

In [13]:
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

In [14]:
print("[INFO] Compiling model")
opt = SGD(lr=0.05)
model = MiniVGGNet.build(width=64, height=64, depth=3, classes=len(classNames))
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy'])

[INFO] Loading images


In [ ]:
print("[INFO] Training network")
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=32, epochs=100, verbose=1)

[INFO] Training network
Train on 1020 samples, validate on 340 samples
Epoch 1/100
1020/1020 [==============================] - 38s 37ms/step - loss: 3.1835 - acc: 0.2265 - val_loss: 3.6956 - val_acc: 0.0912
Epoch 2/100
1020/1020 [==============================] - 37s 36ms/step - loss: 2.0527 - acc: 0.4176 - val_loss: 2.5613 - val_acc: 0.2088
Epoch 3/100
1020/1020 [==============================] - 37s 36ms/step - loss: 1.4152 - acc: 0.5657 - val_loss: 2.1297 - val_acc: 0.3147
Epoch 4/100
1020/1020 [==============================] - 37s 36ms/step - loss: 1.0461 - acc: 0.6686 - val_loss: 2.6437 - val_acc: 0.2029
Epoch 5/100
1020/1020 [==============================] - 38s 38ms/step - loss: 0.9058 - acc: 0.6931 - val_loss: 2.8419 - val_acc: 0.2412
Epoch 6/100
1020/1020 [==============================] - 37s 36ms/step - loss: 0.6084 - acc: 0.8010 - val_loss: 2.2847 - val_acc: 0.3412
Epoch 7/100
1020/1020 [==============================] - 36s 36ms/step - loss: 0.4807 - acc: 0.8343 - val_l

In [ ]:
print("[INFO] evaluating network")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=classNames))


In [ ]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 100), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, 100), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, 100), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, 100), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.show()